In [ ]:
from IPython.core.debugger import set_trace

import numpy as np
import cv2
import glob
import pickle
from utilities import *

dist_pickle = pickle.load(open('camera_cal/calibration.p','rb'))
mtx = dist_pickle['mtx']
dist = dist_pickle['dist']

In [ ]:
distorted_img = cv2.imread('./test_images/test1.jpg')
undistorted_img= cv2.undistort(distorted_img, mtx, dist, None, mtx)
cv2.imwrite('./output_images/undistorted.jpg', undistorted_img)

In [ ]:
preprocessed_img = preprocess(undistorted_img, mtx, dist)
cv2.imwrite('./output_images/preprocessed.jpg', preprocessed_img)

In [ ]:
import matplotlib.pyplot as plt
straight_img = cv2.imread("./test_images/straight_lines1.jpg")

straight_img = cv2.cvtColor(straight_img, cv2.COLOR_BGR2RGB)
line_color = (0,255,0)
lineThickness = 2
img_size = (straight_img.shape[1], straight_img.shape[0])

pt1 = (int(img_size[0]*0.544), int(img_size[1]*0.63))
pt2 = (int(img_size[0]*0.87), int(img_size[1]))
pt3 = (int(img_size[0]*0.14),img_size[1])
pt4 = (int(img_size[0]*0.458),int(img_size[1]*0.63))

perspective_img, Minv, undist = calc_perspective(straight_img, mtx, dist, img_size, preprocess_img=False)

original_img = cv2.line(np.copy(undist), pt1, pt2, line_color, lineThickness)
original_img = cv2.line(original_img, pt2, pt3, line_color, lineThickness)
original_img = cv2.line(original_img, pt3, pt4, line_color, lineThickness)
original_img = cv2.line(original_img, pt1, pt4, line_color, lineThickness)

fig = plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.title('original')
plt.imshow(original_img)




offset = img_size[0] * 0.23
pt1 = (int(img_size[0]-offset), 0)
pt2 = (int(img_size[0]-offset), int(img_size[1]))
pt3 = (int(offset),img_size[1])
pt4 = (int(offset),int(0))

lineThickness = 10
perspective_img = cv2.line(perspective_img, pt1, pt2, line_color, lineThickness)
perspective_img = cv2.line(perspective_img, pt2, pt3, line_color, lineThickness)
perspective_img = cv2.line(perspective_img, pt3, pt4, line_color, lineThickness)
perspective_img = cv2.line(perspective_img, pt1, pt4, line_color, lineThickness) 


plt.subplot(1, 2, 2)
plt.title('perspective')
plt.imshow(perspective_img)
plt.savefig('./output_images/perspective_transform.jpg')

In [ ]:
images = glob.glob('./test_images/test*.jpg')
format = './test_images/output/{1}{0}.jpg'

for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result, preprocess_image, warped, road_perspective = process_image(img=img_rgb, mtx=mtx,dist=dist, return_debug_images=True)   
    
    result = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)
    result_write_name = format.format('tracked',idx)
    cv2.imwrite(result_write_name,result)    
        
    preprocess_write_name =  format.format('preprocess', idx) 
    cv2.imwrite(preprocess_write_name, preprocess_image)
    
    cv2.imwrite(format.format('warped', idx), warped)
    
    cv2.imwrite(format.format('original', idx), img)        
    
    cv2.imwrite(format.format('road perspective', idx), road_perspective)
    
print('...')